In [29]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import torch.nn as nn	# 各种层类型的实现
import torch.nn.functional as F	# 各中层函数的实现，与层类型对应，如：卷积函数、池化函数、归一化函数等等
import torch.optim as optim	# 实现各种优化算法的包
from torchvision import datasets, transforms
import numpy as np

import argparse
import sys


import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
from grid_data import GridData

from compute_mpe import CircuitMPE
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')
import gurobipy
cmpe = CircuitMPE('/root/PySDD/notebooks/ml100.vtree', '/root/PySDD/notebooks/ml100.sdd')
class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=100
        self.card=20
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
        self.config='scad.yaml'
        self.shared='store_true'
        self.env_id='MsPacmanNoFrameskip-v4'
        self.cuda='store_true'
        self.seed=0
args=para()
legalList={0: [46], 1: [59], 2: [53, 63, 86], 3: [17, 22, 23, 24, 26, 34, 51, 61, 66, 74, 81, 85, 87, 89, 90, 96], 4: [24, 26, 37, 71, 83, 87, 94, 99], 5: [96], 6: [38], 7: [85], 8: [19, 64], 9: [], 10: [], 11: [], 12: [17, 79, 81], 13: [17, 23, 24, 25, 26, 30, 36, 42, 54, 56, 61, 66, 68, 72, 77, 81, 84, 89, 92, 95], 14: [], 15: [], 16: [17, 24, 25, 27, 35, 39, 42, 46, 48, 56, 61, 66, 72, 76, 81, 84, 89], 17: [3, 12, 13, 16, 24, 25, 27, 29, 35, 39, 42, 46, 51, 52, 54, 56, 61, 66, 72, 76, 77, 79, 81, 85, 89, 92, 96], 18: [], 19: [8, 37, 64, 70, 91], 20: [], 21: [], 22: [3, 30, 61, 66, 78, 81, 85, 90, 96], 23: [3, 13, 61, 66, 89], 24: [3, 4, 13, 16, 17, 25, 26, 27, 31, 34, 38, 42, 51, 54, 56, 61, 66, 68, 72, 74, 76, 77, 81, 84, 87, 89, 99], 25: [13, 16, 17, 24, 27, 35, 36, 39, 40, 42, 48, 51, 52, 54, 56, 61, 66, 68, 72, 76, 77, 81, 84, 89, 92], 26: [3, 4, 13, 24, 34, 42, 54, 66, 74, 77, 81, 83, 92, 97], 27: [16, 17, 24, 25, 35, 39, 46, 51, 52, 61, 68, 76, 87, 99], 28: [], 29: [17, 35, 56, 66, 72, 79, 81, 89], 30: [13, 22, 54, 61, 66, 73, 75, 78, 81, 95], 31: [24, 46, 51, 52, 61, 76, 79], 32: [], 33: [], 34: [3, 24, 26, 54, 66, 71, 74, 77, 81, 84, 94], 35: [16, 17, 25, 27, 29, 36, 39, 40, 46, 48, 52, 56, 61, 66, 72, 76, 77, 81, 89], 36: [13, 25, 35, 42, 56, 57, 66, 77, 81, 84, 89, 92], 37: [4, 19, 51, 64, 87, 99], 38: [6, 24, 51, 63, 67, 99], 39: [16, 17, 25, 27, 35, 48, 56, 61, 76, 77, 81, 85, 89], 40: [25, 35], 41: [], 42: [13, 16, 17, 24, 25, 26, 36, 54, 56, 61, 66, 68, 72, 76, 77, 81, 84, 89, 92], 43: [], 44: [51, 74], 45: [], 46: [0, 16, 17, 27, 31, 35, 51, 52, 61, 76, 96], 47: [], 48: [16, 25, 35, 39, 56, 61, 66, 77, 89], 49: [], 50: [], 51: [3, 17, 24, 25, 27, 31, 37, 38, 44, 46, 61, 68, 76, 87, 94, 99], 52: [17, 25, 27, 31, 35, 46, 61, 72, 76, 79, 81, 89], 53: [2, 70, 86, 93], 54: [13, 17, 24, 25, 26, 30, 34, 42, 56, 61, 66, 68, 72, 77, 81, 84, 89, 92, 95], 55: [], 56: [13, 16, 17, 24, 25, 29, 35, 36, 39, 42, 48, 54, 61, 66, 77, 81, 84, 89, 92], 57: [36], 58: [], 59: [1], 60: [], 61: [3, 13, 16, 17, 22, 23, 24, 25, 27, 30, 31, 35, 39, 42, 46, 48, 51, 52, 54, 56, 66, 72, 76, 77, 79, 81, 84, 85, 87, 89, 90, 96], 62: [], 63: [2, 38], 64: [8, 19, 37, 74, 87, 91], 65: [], 66: [3, 13, 16, 17, 22, 23, 24, 25, 26, 29, 30, 34, 35, 36, 42, 48, 54, 56, 61, 72, 73, 74, 76, 77, 81, 84, 85, 89, 92], 67: [38], 68: [13, 24, 25, 27, 42, 51, 54, 87, 99], 69: [], 70: [19, 53], 71: [4, 34, 74], 72: [13, 16, 17, 24, 25, 29, 35, 42, 52, 54, 61, 66, 76, 79, 81, 89], 73: [30, 66, 78, 84], 74: [3, 24, 26, 34, 44, 64, 66, 71, 77, 83], 75: [30], 76: [16, 17, 24, 25, 27, 31, 35, 39, 42, 46, 51, 52, 61, 66, 72, 79, 81, 89, 96], 77: [13, 17, 24, 25, 26, 34, 35, 36, 39, 42, 48, 54, 56, 61, 66, 74, 81, 84, 89, 92], 78: [22, 30, 73, 85, 90], 79: [12, 17, 29, 31, 52, 61, 72, 76, 81, 89], 80: [], 81: [3, 12, 13, 16, 17, 22, 24, 25, 26, 29, 30, 34, 35, 36, 39, 42, 52, 54, 56, 61, 66, 72, 76, 77, 79, 84, 85, 89, 92, 95], 82: [], 83: [4, 26, 74], 84: [13, 16, 24, 25, 34, 36, 42, 54, 56, 61, 66, 73, 77, 81, 89], 85: [3, 7, 17, 22, 39, 61, 66, 78, 81, 89, 90], 86: [2, 53, 88, 91], 87: [3, 4, 24, 27, 37, 51, 61, 64, 68, 91, 94, 99], 88: [86], 89: [3, 13, 16, 17, 23, 24, 25, 29, 35, 36, 39, 42, 48, 52, 54, 56, 61, 66, 72, 76, 77, 79, 81, 84, 85, 92], 90: [3, 22, 61, 78, 85], 91: [19, 64, 86, 87, 94], 92: [13, 17, 25, 26, 36, 42, 54, 56, 66, 77, 81, 89, 95], 93: [53], 94: [4, 34, 51, 87, 91, 99], 95: [13, 30, 54, 81, 92], 96: [3, 5, 17, 22, 46, 61, 76], 97: [26], 98: [], 99: [4, 24, 27, 37, 38, 51, 68, 87, 94]}
def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(x.sum()-args.card)/args.card/2
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()
    # 创建模型
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    # 更新变量环境
    MODEL.update()

    # 创建目标函数
    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)

    # 创建约束条件
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    # 执行最优化
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()
    # 创建模型
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    # 更新变量环境
    MODEL.update()

    # 创建目标函数
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    # 创建约束条件
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    # 执行最优化
    
    print('??')
    MODEL.optimize()
    print('??')
    return np.array(MODEL.x)

In [47]:
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np

MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 20000

INITIAL_ACCOUNT_BALANCE = 10000
legalList={0: [46], 1: [59], 2: [53, 63, 86], 3: [17, 22, 23, 24, 26, 34, 51, 61, 66, 74, 81, 85, 87, 89, 90, 96], 4: [24, 26, 37, 71, 83, 87, 94, 99], 5: [96], 6: [38], 7: [85], 8: [19, 64], 9: [], 10: [], 11: [], 12: [17, 79, 81], 13: [17, 23, 24, 25, 26, 30, 36, 42, 54, 56, 61, 66, 68, 72, 77, 81, 84, 89, 92, 95], 14: [], 15: [], 16: [17, 24, 25, 27, 35, 39, 42, 46, 48, 56, 61, 66, 72, 76, 81, 84, 89], 17: [3, 12, 13, 16, 24, 25, 27, 29, 35, 39, 42, 46, 51, 52, 54, 56, 61, 66, 72, 76, 77, 79, 81, 85, 89, 92, 96], 18: [], 19: [8, 37, 64, 70, 91], 20: [], 21: [], 22: [3, 30, 61, 66, 78, 81, 85, 90, 96], 23: [3, 13, 61, 66, 89], 24: [3, 4, 13, 16, 17, 25, 26, 27, 31, 34, 38, 42, 51, 54, 56, 61, 66, 68, 72, 74, 76, 77, 81, 84, 87, 89, 99], 25: [13, 16, 17, 24, 27, 35, 36, 39, 40, 42, 48, 51, 52, 54, 56, 61, 66, 68, 72, 76, 77, 81, 84, 89, 92], 26: [3, 4, 13, 24, 34, 42, 54, 66, 74, 77, 81, 83, 92, 97], 27: [16, 17, 24, 25, 35, 39, 46, 51, 52, 61, 68, 76, 87, 99], 28: [], 29: [17, 35, 56, 66, 72, 79, 81, 89], 30: [13, 22, 54, 61, 66, 73, 75, 78, 81, 95], 31: [24, 46, 51, 52, 61, 76, 79], 32: [], 33: [], 34: [3, 24, 26, 54, 66, 71, 74, 77, 81, 84, 94], 35: [16, 17, 25, 27, 29, 36, 39, 40, 46, 48, 52, 56, 61, 66, 72, 76, 77, 81, 89], 36: [13, 25, 35, 42, 56, 57, 66, 77, 81, 84, 89, 92], 37: [4, 19, 51, 64, 87, 99], 38: [6, 24, 51, 63, 67, 99], 39: [16, 17, 25, 27, 35, 48, 56, 61, 76, 77, 81, 85, 89], 40: [25, 35], 41: [], 42: [13, 16, 17, 24, 25, 26, 36, 54, 56, 61, 66, 68, 72, 76, 77, 81, 84, 89, 92], 43: [], 44: [51, 74], 45: [], 46: [0, 16, 17, 27, 31, 35, 51, 52, 61, 76, 96], 47: [], 48: [16, 25, 35, 39, 56, 61, 66, 77, 89], 49: [], 50: [], 51: [3, 17, 24, 25, 27, 31, 37, 38, 44, 46, 61, 68, 76, 87, 94, 99], 52: [17, 25, 27, 31, 35, 46, 61, 72, 76, 79, 81, 89], 53: [2, 70, 86, 93], 54: [13, 17, 24, 25, 26, 30, 34, 42, 56, 61, 66, 68, 72, 77, 81, 84, 89, 92, 95], 55: [], 56: [13, 16, 17, 24, 25, 29, 35, 36, 39, 42, 48, 54, 61, 66, 77, 81, 84, 89, 92], 57: [36], 58: [], 59: [1], 60: [], 61: [3, 13, 16, 17, 22, 23, 24, 25, 27, 30, 31, 35, 39, 42, 46, 48, 51, 52, 54, 56, 66, 72, 76, 77, 79, 81, 84, 85, 87, 89, 90, 96], 62: [], 63: [2, 38], 64: [8, 19, 37, 74, 87, 91], 65: [], 66: [3, 13, 16, 17, 22, 23, 24, 25, 26, 29, 30, 34, 35, 36, 42, 48, 54, 56, 61, 72, 73, 74, 76, 77, 81, 84, 85, 89, 92], 67: [38], 68: [13, 24, 25, 27, 42, 51, 54, 87, 99], 69: [], 70: [19, 53], 71: [4, 34, 74], 72: [13, 16, 17, 24, 25, 29, 35, 42, 52, 54, 61, 66, 76, 79, 81, 89], 73: [30, 66, 78, 84], 74: [3, 24, 26, 34, 44, 64, 66, 71, 77, 83], 75: [30], 76: [16, 17, 24, 25, 27, 31, 35, 39, 42, 46, 51, 52, 61, 66, 72, 79, 81, 89, 96], 77: [13, 17, 24, 25, 26, 34, 35, 36, 39, 42, 48, 54, 56, 61, 66, 74, 81, 84, 89, 92], 78: [22, 30, 73, 85, 90], 79: [12, 17, 29, 31, 52, 61, 72, 76, 81, 89], 80: [], 81: [3, 12, 13, 16, 17, 22, 24, 25, 26, 29, 30, 34, 35, 36, 39, 42, 52, 54, 56, 61, 66, 72, 76, 77, 79, 84, 85, 89, 92, 95], 82: [], 83: [4, 26, 74], 84: [13, 16, 24, 25, 34, 36, 42, 54, 56, 61, 66, 73, 77, 81, 89], 85: [3, 7, 17, 22, 39, 61, 66, 78, 81, 89, 90], 86: [2, 53, 88, 91], 87: [3, 4, 24, 27, 37, 51, 61, 64, 68, 91, 94, 99], 88: [86], 89: [3, 13, 16, 17, 23, 24, 25, 29, 35, 36, 39, 42, 48, 52, 54, 56, 61, 66, 72, 76, 77, 79, 81, 84, 85, 92], 90: [3, 22, 61, 78, 85], 91: [19, 64, 86, 87, 94], 92: [13, 17, 25, 26, 36, 42, 54, 56, 66, 77, 81, 89, 95], 93: [53], 94: [4, 34, 51, 87, 91, 99], 95: [13, 30, 54, 81, 92], 96: [3, 5, 17, 22, 46, 61, 76], 97: [26], 98: [], 99: [4, 24, 27, 37, 38, 51, 68, 87, 94]}

def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(x.sum()-args.card)/args.card/2

class Env(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        super(Env, self).__init__()
        self.episodes=0
        self.action_space =  spaces.Tuple([spaces.Box(low=0,high=1,shape=(args.solDim,1))])

        # Prices contains the OHCL values for the last five prices
        self.observation_space =  spaces.Tuple([spaces.Box(low=0,high=1,shape=(args.solDim,1))])

    def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        return None

    def step(self, action,probs):
        # Execute one time step within the environment
        self.episodes+=1
        yu=torch.unbind(torch.from_numpy(np.array(action)))
        xu = torch.unbind(torch.from_numpy(np.array([-1]*args.solDim).astype(np.float32)))
        wmc = cmpe.get_torch_ac([[1.0 - ny,ny] for ny in yu + xu]) 
        reward =-(100*(sum(action)-20)**2-wmc)# 50000*(1/(1+np.exp(-self.episodes))-0.5)*torch.log(torch.mean(wmc)).float()+
        done = 1
        obs = self._next_observation()

        return obs, reward, done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        return torch.from_numpy(np.array([-0.0]*args.solDim)).float()

    def render(self, mode='human', close=False):
        pass

In [56]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = Env()

state_size = args.solDim
action_size =args.solDim
lr = 0.0001

class Actor(nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 16)
        self.linear2 = nn.Linear(16, 32)
        self.linear3 = nn.Linear(32, args.solDim)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = F.sigmoid(output)#Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 16)
        self.linear2 = nn.Linear(16, 32)
        self.linear3 = nn.Linear(32,1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        
        view = self.linear3(output)
        return view


def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

n_iters=1000


#if os.path.exists('model/actor.pkl'):
 #   actor = torch.load('model/actor.pkl')
  #  print('Actor Model loaded')

actor = Actor(state_size, action_size).to(device)
def weights_init(mod):
    """设计初始化函数"""
    classname=mod.__class__.__name__
    # 返回传入的module类型
    print(classname)
    if classname.find('Conv')!= -1:    #这里的Conv和BatchNnorm是torc.nn里的形式
        mod.weight.data.normal_(-0.2,0.2)
    elif classname.find('BatchNorm')!= -1:
        mod.weight.data.normal_(1.0,0.2) #bn层里初始化γ，服从（1，0.02）的正态分布
        mod.bias.data.fill_(0)  #bn层里初始化β，默认为0

actor.apply(weights_init)
#if os.path.exists('model/critic.pkl'):
 #   critic = torch.load('model/critic.pkl')
  #  print('Critic Model loaded')

critic = Critic(state_size, action_size).to(device)
critic.apply(weights_init)
optimizerA = optim.Adam(actor.parameters())
optimizerC = optim.Adam(critic.parameters())
for iter in range(n_iters):
    state = env.reset()
    log_probs = []
    values = []
    rewards = []
    masks = []
    entropy = 0
    env.reset()

    for i in count():
        env.render()
        state = torch.FloatTensor(state).to(device)
        probs, value = actor(state), critic(state)
        b= torch.distributions.Bernoulli(probs)
        action =b.sample()
        log_prob = b.entropy()

        next_state, reward, done, _ = env.step(action,probs)

        entropy +=log_prob.mean()
        print('entropy+reward+feai',entropy,reward,checkFea(action),sum(action))
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
        masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

        state = next_state

        if done:
            #print('Iteration: {}, Score: {}'.format(iter, i))
            break


    next_value = critic(probs)
    returns = compute_returns(next_value, rewards, masks)

    log_probs = torch.cat(log_probs)
    returns = torch.cat(returns).detach()
    values = torch.cat(values)
#-(log_probs * advantage.detach()).mean()
    advantage = returns - values
    yu=torch.unbind(torch.from_numpy(np.array(probs.detach())))
    xu = torch.unbind(torch.from_numpy(np.array([-1]*args.solDim).astype(np.float32)))
    wmc = cmpe.get_torch_ac([[1.0 - ny,ny] for ny in yu + xu]) 
    actor_loss  = -(log_probs*reward).mean()#.requires_grad_()#-(log_probs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()
    print('loss',actor_loss)
    optimizerA.zero_grad()
    optimizerC.zero_grad()
    actor_loss.backward()
    critic_loss.backward()
    optimizerA.step()
    optimizerC.step()
torch.save(actor, 'model/actor.pkl')
torch.save(critic, 'model/critic.pkl')
env.close()


Linear
Linear
Linear
Actor
Linear
Linear
Linear
Critic
entropy+reward+feai tensor(0.6916, grad_fn=<AddBackward0>) tensor([-78400.]) tensor(0.7685) tensor(48.)
loss tensor(54221.9648, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6915, grad_fn=<AddBackward0>) tensor([-84100.]) tensor(0.8236) tensor(49.)
loss tensor(58157.8359, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6915, grad_fn=<AddBackward0>) tensor([-84100.]) tensor(0.8113) tensor(49.)
loss tensor(58151.2500, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6914, grad_fn=<AddBackward0>) tensor([-84100.]) tensor(0.8442) tensor(49.)
loss tensor(58144.3633, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6913, grad_fn=<AddBackward0>) tensor([-72900.]) tensor(0.8216) tensor(47.)
loss tensor(50394.7500, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6912, grad_fn=<AddBackward0>) tensor([-115600.]) tensor(0.9966) tensor(54.)
loss tensor(79902.3438, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6911, gra

loss tensor(35608.7148, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6721, grad_fn=<AddBackward0>) tensor([-84100.]) tensor(0.8318) tensor(49.)
loss tensor(56525.8594, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6711, grad_fn=<AddBackward0>) tensor([-62500.]) tensor(0.7483) tensor(45.)
loss tensor(41942.1992, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6700, grad_fn=<AddBackward0>) tensor([-122500.]) tensor(1.0654) tensor(55.)
loss tensor(82074.4375, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6688, grad_fn=<AddBackward0>) tensor([-115600.]) tensor(0.9555) tensor(54.)
loss tensor(77317.4219, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6676, grad_fn=<AddBackward0>) tensor([-52900.]) tensor(0.6408) tensor(43.)
loss tensor(35316.0234, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6663, grad_fn=<AddBackward0>) tensor([-57600.]) tensor(0.6712) tensor(44.)
loss tensor(38381.4883, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.6651, grad_fn=<A

loss tensor(38302.0391, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.4471, grad_fn=<AddBackward0>) tensor([-108900.]) tensor(0.9414) tensor(53.)
loss tensor(48691.8242, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.4387, grad_fn=<AddBackward0>) tensor([-144400.]) tensor(1.1829) tensor(58.)
loss tensor(63353.0938, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.4301, grad_fn=<AddBackward0>) tensor([-96100.]) tensor(0.9134) tensor(51.)
loss tensor(41333.7812, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.4215, grad_fn=<AddBackward0>) tensor([-184900.]) tensor(1.2901) tensor(63.)
loss tensor(77938.0781, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.4126, grad_fn=<AddBackward0>) tensor([-102400.]) tensor(0.9466) tensor(52.)
loss tensor(42247.7500, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.4037, grad_fn=<AddBackward0>) tensor([-152100.]) tensor(1.1216) tensor(59.)
loss tensor(61399.8359, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.3946, grad_fn

loss tensor(10837.0400, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0808, grad_fn=<AddBackward0>) tensor([-144400.]) tensor(1.1295) tensor(58.)
loss tensor(11668.1777, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0781, grad_fn=<AddBackward0>) tensor([-115600.]) tensor(1.0103) tensor(54.)
loss tensor(9026.9551, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0755, grad_fn=<AddBackward0>) tensor([-136900.]) tensor(1.0962) tensor(57.)
loss tensor(10337.4336, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0730, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(9465.2061, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0707, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0726) tensor(56.)
loss tensor(9158.1152, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0684, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(8864.3037, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0662, grad_fn=<A

entropy+reward+feai tensor(0.0208, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(2691.1287, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0204, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(2644.7388, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0201, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(2599.6313, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0197, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(2555.7441, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0194, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(2513.0518, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0191, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(2471.4910, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0188, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) t

...

loss tensor(38.0592, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(37.9189, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(37.7822, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(37.6410, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(37.5099, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(37.3662, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(37.2275, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0003, grad_fn=<AddBackward0>) ten

loss tensor(31.3213, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(31.2257, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(31.1113, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(31.0202, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(30.9052, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(30.8004, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(30.7009, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) ten

loss tensor(26.1922, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(26.1128, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(26.0252, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(25.9514, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(25.8677, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(25.7961, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(25.7029, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) ten

loss tensor(22.1970, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(22.1395, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(22.0667, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(22.0066, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(21.9511, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(21.8803, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(21.8172, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0002, grad_fn=<AddBackward0>) ten

loss tensor(18.9785, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0001, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(18.9365, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0001, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(18.8829, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0001, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(18.8366, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0001, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(18.7826, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0001, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(18.7267, grad_fn=<NegBackward>)
entropy+reward+feai tensor(0.0001, grad_fn=<AddBackward0>) tensor([-129600.]) tensor(1.0712) tensor(56.)
loss tensor(18.6782, grad_fn=<NegBackward>)


In [46]:
sum(probs)

tensor(19.9999, grad_fn=<AddBackward0>)

In [338]:
 F.softmax(torch.tensor(0.8),dim=-1)

tensor(1.)

In [ ]:
np.array((1,2))